In [1]:
# coding: utf-8

import matplotlib.pyplot as plt
import pandas as pd 
import sys 
import matplotlib 
import functools
import numpy as np
import time
import multiprocessing  
import random
#import spectrum
import KSTEST
import itertools
import os.path

from pylab import plot
from scipy import signal
from pandas import DataFrame, read_csv
from scipy import stats
#from spectrum import *
from datetime import timedelta, date, datetime

from multiprocessing import Pool

#Clean Cells That Have only 0 values for 
def CleanNullCells(ZoneDataFrame):
    for kpi in column_kpiselection:
        for cell in ZoneDataFrame['CellName'].unique():
            CelluleData = ZoneDataFrame.loc[ZoneDataFrame['CellName']==cell][['DAY_',kpi]]
            if(CelluleData[kpi] == 0).all():
                  ZoneDataFrame = ZoneDataFrame.loc[ZoneDataFrame['CellName'] != cell]
    return(ZoneDataFrame) 




def KSTestPerDay(cell1,cell2,kpi):
    cell1_kpi = alldata_kpis.loc[alldata_kpis['CellName']==cell1][['DAY_',kpi]]
    cell1_kpi['DAY_'] = pd.to_datetime(cell1_kpi['DAY_'],infer_datetime_format=True)    
    cell2_kpi = alldata_kpis.loc[alldata_kpis['CellName']==cell2][['DAY_',kpi]]
    cell2_kpi['DAY_'] = pd.to_datetime(cell2_kpi['DAY_'],infer_datetime_format=True)
    
    KS = pd.DataFrame()
    for day in np.array([datetime(2018,8,7)+timedelta(days=i) for i in range(0,22)]):
        dayvalue = pd.DataFrame()
        dayvalue['Day'] = [day]
        mask1 = (cell1_kpi['DAY_'] >=day) & (cell1_kpi['DAY_'] <day+timedelta(days=1))
        mask2 = (cell2_kpi['DAY_'] >=day) & (cell2_kpi['DAY_'] <day+timedelta(days=1))

        dayvalue['pvalue'] = [stats.ks_2samp(cell1_kpi.loc[mask1][kpi],cell2_kpi.loc[mask2][kpi])[1]]
        KS = KS.append(dayvalue,ignore_index = True)
    return(KS)




def KSTestPerWeekDay(cell1,cell2,kpi):
    cell1_kpi = alldata_kpis.loc[alldata_kpis['CellName']==cell1][['DAY_',kpi]]   
    cell1_kpi['DAY_'] = pd.to_datetime(cell1_kpi['DAY_'],infer_datetime_format=True)
    cell2_kpi = alldata_kpis.loc[alldata_kpis['CellName']==cell2][['DAY_',kpi]]
    cell2_kpi['DAY_'] = pd.to_datetime(cell2_kpi['DAY_'],infer_datetime_format=True)
  
    KS = pd.DataFrame()
    for day in range(0,7):
        dayvalue = pd.DataFrame()
        dayvalue['Day'] = [day]
        mask1 = (cell1_kpi['DAY_'].dt.dayofweek==day)
        mask2 = (cell2_kpi['DAY_'].dt.dayofweek==day)
        dayvalue['pvalue'] = [stats.ks_2samp(cell1_kpi.loc[mask1][kpi],cell2_kpi.loc[mask2][kpi])[1]]
        KS = KS.append(dayvalue,ignore_index = True)
    return(KS)

def KSTestAllCellsPerDayAverage(ZoneDataFrame,kpi):
    KS = pd.DataFrame()
    alreadyDoneCouples = []
    for cell1 in ZoneDataFrame['CellName'].unique():
        for cell2 in ZoneDataFrame['CellName'].unique():
            if cell1 == cell2:
                KS.at[str(cell1),str(cell2)] = float(1)
            elif [cell2,cell1] in alreadyDoneCouples:
                KS.at[str(cell1),str(cell2)] = KS.loc[str(cell2)][str(cell1)]
            else:
                avg = KSTestPerDay(cell1,cell2,kpi)['pvalue'].mean()
                KS.at[cell1,cell2] = avg
                alreadyDoneCouples.append([cell1,cell2])
    return(KS)



def KSTestAllCellsPerDayStd(ZoneDataFrame,kpi):
    KS = pd.DataFrame()
    alreadyDoneCouples = []
    for cell1 in ZoneDataFrame['CellName'].unique():
        for cell2 in ZoneDataFrame['CellName'].unique():
            if cell1 == cell2:
                KS.loc[str(cell1),str(cell2)] = float(0)
            elif [cell2,cell1] in alreadyDoneCouples:
                KS.at[str(cell1),str(cell2)] = KS.loc[str(cell2)][str(cell1)]
            else:
                std = KSTestPerDay(cell1,cell2,kpi)['pvalue'].std()
                KS.at[str(cell1),str(cell2)] = float(std)
                alreadyDoneCouples.append([cell1,cell2])
    return(KS)



def KSTestAllCellsPerWeekDayAverage(ZoneDataFrame,kpi):
    KS = pd.DataFrame()
    alreadyDoneCouples = []
    for cell1 in ZoneDataFrame['CellName'].unique():
        for cell2 in ZoneDataFrame['CellName'].unique():
            if cell1 == cell2:
                KS.at[str(cell1),str(cell2)] = float(1)
            elif [cell2,cell1] in alreadyDoneCouples:
                KS.at[str(cell1),str(cell2)] = KS.loc[str(cell2)][str(cell1)]
            else:
                avg = KSTestPerWeekDay(cell1,cell2,kpi)['pvalue'].mean()
                KS.at[cell1,cell2] = avg
                alreadyDoneCouples.append([cell1,cell2])
    return(KS)


def KSTestAllCellsPerWeekDayStd(ZoneDataFrame,kpi):
    KS = pd.DataFrame()
    alreadyDoneCouples = []
    for cell1 in ZoneDataFrame['CellName'].unique():
        for cell2 in ZoneDataFrame['CellName'].unique():
            if cell1 == cell2:
                KS.loc[str(cell1),str(cell2)] = float(0)
            elif [cell2,cell1] in alreadyDoneCouples:
                KS.at[str(cell1),str(cell2)] = KS.loc[str(cell2)][str(cell1)]
            else:
                std = KSTestPerWeekDay(cell1,cell2,kpi)['pvalue'].std()
                KS.at[str(cell1),str(cell2)] = float(std)
                alreadyDoneCouples.append([cell1,cell2])
    return(KS)



def getIdenticallyDistributedUnits(KSTestMatrix, alreadyDeletedCells = []): 
    count = 0
    totalpvalues = float(0)
    ToDeleteCell = 0
    for cell in KSTestMatrix.index.unique():
        A = np.array(KSTestMatrix[cell].values)
        A = A[np.where(A<0.1)]
        if len(A)>0:
            if count < len(A):
                ToDeleteCell = cell 
                count = len(A)
                totalpvalues = sum(A)
            elif (count == len(A)) & (totalpvalues < float(sum(A))):
                ToDeleteCell = cell 
                count = len(A)
                totalpvalues = sum(A)
    if ToDeleteCell == 0:
        return([KSTestMatrix,alreadyDeletedCells])
    else:
        alreadyDeletedCells.append(ToDeleteCell)
        FiltredMatrix = KSTestMatrix.drop([ToDeleteCell], axis = 0)
        FiltredMatrix = FiltredMatrix.drop([ToDeleteCell], axis = 1)
        return(getIdenticallyDistributedUnits(FiltredMatrix,alreadyDeletedCells))
        

def getIdenticallyDistributedClusters(KSTestMatrix, partialResults=[]):
    Cluster = getIdenticallyDistributedUnits(KSTestMatrix,[])[0]
    DeletedCells = getIdenticallyDistributedUnits(KSTestMatrix,[])[1]
    partialResults.append(Cluster)
    if Cluster.size!=KSTestMatrix.size:
        Reste = KSTestMatrix.loc[DeletedCells][DeletedCells]
        return(getIdenticallyDistributedClusters(Reste,partialResults))
    else:
        return(partialResults)


def generateLabeledDataFromClusters(ClusterList,kpi):
    LabeledData = pd.DataFrame()
    Label = 0
    for cluster in ClusterList:
        for clustercell in cluster.index:
            LabeledData.loc[str(clustercell),str(kpi)+' Label'] = Label
        Label = Label + 1
    return(LabeledData)


def generateFeaturesFromCellList(CellList):
    FeaturesData = pd.DataFrame()
    for cell in CellList:
        FeaturesData.loc[cell,'Site'] = int(cell[1:len(cell)-1])
        if cell.startswith('D'):
            FeaturesData.loc[cell,'Tec'] = 0
            if cell[-1] =='A':
                FeaturesData.loc[cell,'Azimuth'] = 0
            elif cell[-1] =='B':
                FeaturesData.loc[cell,'Azimuth'] = 1
            elif cell[-1] =='C':
                FeaturesData.loc[cell,'Azimuth'] = 2    
        elif cell.startswith('L'):
            FeaturesData.loc[cell,'Tec'] = 1
            if cell[-1] =='A':
                FeaturesData.loc[cell,'Azimuth'] = 0
            elif cell[-1] =='B':
                FeaturesData.loc[cell,'Azimuth'] = 1
            elif cell[-1] =='C':
                FeaturesData.loc[cell,'Azimuth'] = 2
        elif cell.startswith('T'):
            FeaturesData.loc[cell,'Tec'] = 2
            if cell[-1] =='A':
                FeaturesData.loc[cell,'Azimuth'] = 0
            elif cell[-1] =='B':
                FeaturesData.loc[cell,'Azimuth'] = 1
            elif cell[-1] =='C':
                FeaturesData.loc[cell,'Azimuth'] = 2
    return(FeaturesData)


def generateKpiLabeledDataFromExcelKSTEST(excelFileLocation, kpiList):
    allKpiDataFrame = pd.DataFrame()
    i = 0
    for kpi in kpiList:
        kpiKSTEST = pd.read_excel(excelFileLocation, sheet_name = kpi)
        kpiClusters = getIdenticallyDistributedClusters(kpiKSTEST,[])
        kpiLabeledData = generateLabeledDataFromClusters(kpiClusters,kpi)
        if i == 0:
            cellFeatures = generateFeaturesFromCellList(kpiKSTEST.index)
            allKpiDataFrame = cellFeatures.join(kpiLabeledData)
            i = i+1
        else:
            allKpiDataFrame = allKpiDataFrame.join(kpiLabeledData) 
    return(allKpiDataFrame)

def generateKpiLabeledDataFromExcelKSTEST(excelFileLocation, kpiList):
    allKpiDataFrame = pd.DataFrame()
    time_struct={}
    i = 0
    for kpi in kpiList:
        kpiKSTEST = pd.read_excel(excelFileLocation, sheet_name = kpi)
        time_struct[kpi]={}
        time_struct[kpi]["delta_t"]=[]
        start_time = time.time()
        kpiClusters = getIdenticallyDistributedClusters(kpiKSTEST,[])
        end_time = time.time()
        time_struct[kpi]["delta_t"].append(end_time-start_time)
 
        kpiLabeledData = generateLabeledDataFromClusters(kpiClusters,kpi)
        if i == 0:
            cellFeatures = generateFeaturesFromCellList(kpiKSTEST.index)
            allKpiDataFrame = cellFeatures.join(kpiLabeledData)
            i = i+1
        else:
            allKpiDataFrame = allKpiDataFrame.join(kpiLabeledData) 
    return(allKpiDataFrame,time_struct)

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

def KSTestAllCellsPerDayAverageParallel(ZoneDataFrame,kpi):
    C = list(itertools.combinations(ZoneDataFrame['CellName'].unique(),2))
    A = [(ZoneDataFrame, c[0],c[1],kpi) for c in C]
    if __name__ == '__main__':
        from datetime import timedelta, date, datetime
        pool = Pool()
        t = time.time()
        print(time.time()-t)
        results = pool.starmap(KSTEST.KSTestPerDayAverageParallelFunc, A)
    del A
    del C
    return(results)

def buildKSTESTMatrixFromMPTEST(result):
    KS = pd.DataFrame()
    for res in result:
        cell1 = res.index.values[0]
        cell2 = res.columns.values[0]
        pvalue = res.loc[res.index.values[0]][res.columns.values[0]] 
        KS.at[str(cell1),str(cell2)] = pvalue
        KS.at[str(cell2),str(cell1)] = pvalue
    KS = KS.fillna(1)
    return(KS)
        

print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)

Python version 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
Pandas version 0.23.0
Matplotlib version 2.2.2
Python version 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
Pandas version 0.23.0
Matplotlib version 2.2.2


In [2]:
column_selection = ["DAY_","Departement","SiteTheorique","eNodeBName","Secteur","CellName","CQImoyen_Convert","CROZON_DemandesVoix4G",
                    "CROZON_NombredeConnexionsPS","CROZON_SucceseRABData4G","CROZON_TraficDataDL4G","CROZON_SucceseRABPS4G","CoupuresData",
                    "CROZON_SucceseRABVoix4G","CSFB_Tot","CROZON_TDC4GData","CROZON_CoupureseRABData4G_old","CROZON_TDC4GData_Mobilite",
                    "CROZON_DemandesData4G","CROZON_TDC4GDataCluster","Badcoveragedetected_A2_","CROZON_TDC4GDataCluster_Mobilite",
                    "CROZON_CoupureseRABPS4G","CROZON_TDC4GPSCluster_Mobilite","CROZON_TDC4GPS","CQImoyen","CROZON_TDC4GPS_Mobilite",
                    "CROZON_TraficDataUL4G","CROZON_TDC4GPSCluster","ErabrelnormalQCI9","ErabRelQCi5","nbUEVoIPqualityOK","nbUEVoIPqualityNOK",
                    "Nbrtentatives_QCi5__TEAVoix","Nbrtentatives_QCi5__TEAVisio","Nbrtentatives_QCi1_","NbreConn_Voix","ErabrelnormalQCI5",
                    "ErabVoixRelease_Cellule","ErabrelnormalQCI8","NbHOPrepAttIFLB","ErabVoixRelease_Cluster","ErabrelnormalQCI6","NbdropsDataActifseNodeB",
                    "NbreConn_Data","NbHOExeAttIFLB","TBRe_transPUSCH","TBRetransPDSCH","TBtransPUSCH","SCTot","TBtransPDSCH","PingPong_HOinter_HOOscillation",
                    "TraficDataDL","PingPong_HOintra_HOOscillation","TraficDataUL","Traficvoix_Traficvoix_visio","VoIPqualitymeasinsuffstat"]


column_kpiselection = ["CQImoyen_Convert","CROZON_DemandesVoix4G",
                    "CROZON_NombredeConnexionsPS","CROZON_SucceseRABData4G","CROZON_TraficDataDL4G","CROZON_SucceseRABPS4G","CoupuresData",
                    "CROZON_SucceseRABVoix4G","CSFB_Tot","CROZON_TDC4GData","CROZON_TDC4GData_Mobilite",
                    "CROZON_DemandesData4G","CROZON_TDC4GDataCluster","CROZON_TDC4GDataCluster_Mobilite",
                    "CROZON_CoupureseRABPS4G","CROZON_TDC4GPSCluster_Mobilite","CROZON_TDC4GPS","CQImoyen","CROZON_TDC4GPS_Mobilite",
                    "CROZON_TraficDataUL4G","CROZON_TDC4GPSCluster","ErabrelnormalQCI9","ErabRelQCi5","nbUEVoIPqualityOK",
                    "Nbrtentatives_QCi5__TEAVoix","Nbrtentatives_QCi5__TEAVisio","Nbrtentatives_QCi1_","NbreConn_Voix","ErabrelnormalQCI5",
                    "ErabVoixRelease_Cellule","ErabrelnormalQCI8","NbHOPrepAttIFLB","ErabrelnormalQCI6",
                    "NbreConn_Data", "TraficDataDL","TraficDataUL","Traficvoix_Traficvoix_visio"]

In [3]:
########################################
# import des donnees
########################################


start_date = date(2018, 8, 7)
end_date = date(2018, 9, 13)

data_path = 'C:/Users/euripab/Desktop/ML Extraction/Modified Files/'

#Regroupe data of all dates 1
datestring = end_date.strftime("%Y%m%d")
Location1 =  str(data_path)+datestring+'_1.CSV'
date_data = pd.read_csv(Location1, sep=';', encoding='latin-1')
data_1 = pd.read_csv(Location1, sep=';', encoding='latin-1')
data_1  = data_1[column_selection]
data_1 = data_1.replace(r'\N',np.nan)

for single_date in daterange(start_date, end_date):
    datestring = single_date.strftime("%Y%m%d")
    Location1 =  str(data_path)+datestring+'_1.CSV'
    date_data = pd.read_csv(Location1, sep=';', encoding='latin-1')
    date_data = date_data[column_selection]
    date_data = date_data.replace(r'\N',np.nan)
    data_1 = data_1.append(date_data)


#Regroupe data of all dates 2
datestring = end_date.strftime("%Y%m%d")
Location2 =  str(data_path)+datestring+'_2.CSV'
date_data = pd.read_csv(Location2, sep=';', encoding='latin-1')
data_2 = pd.read_csv(Location2, sep=';', encoding='latin-1')
data_2  = data_2[column_selection]
data_2 = data_2.replace(r'\N',np.nan)

for single_date in daterange(start_date, end_date):
    datestring = single_date.strftime("%Y%m%d")
    Location2 =  str(data_path)+datestring+'_2.CSV'
    date_data = pd.read_csv(Location2, sep=';', encoding='latin-1')
    date_data = date_data[column_selection]
    date_data = date_data.replace(r'\N',np.nan)
    data_2 = data_2.append(date_data)
    
    
#Create Final Data File
alldata_kpis = data_1.append(data_2)
del data_1
del data_2
print(alldata_kpis.dtypes)


C:\Users\euripab\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (625,626,627,628) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


DAY_                                object
Departement                          int64
SiteTheorique                       object
eNodeBName                          object
Secteur                             object
CellName                            object
CQImoyen_Convert                    object
CROZON_DemandesVoix4G                int64
CROZON_NombredeConnexionsPS          int64
CROZON_SucceseRABData4G              int64
CROZON_TraficDataDL4G               object
CROZON_SucceseRABPS4G                int64
CoupuresData                         int64
CROZON_SucceseRABVoix4G              int64
CSFB_Tot                             int64
CROZON_TDC4GData                    object
CROZON_CoupureseRABData4G_old        int64
CROZON_TDC4GData_Mobilite           object
CROZON_DemandesData4G                int64
CROZON_TDC4GDataCluster             object
Badcoveragedetected_A2_              int64
CROZON_TDC4GDataCluster_Mobilite    object
CROZON_CoupureseRABPS4G              int64
CROZON_TDC4

In [4]:
print("Format data")
alldata_kpis[column_kpiselection] = alldata_kpis[column_kpiselection].astype(str).apply(lambda x: x.str.replace(',','.'))
print("Force data type")
alldata_kpis[column_kpiselection] = alldata_kpis[column_kpiselection].astype(float)
print("Remove unusefull structure")
alldata_kpis = alldata_kpis.drop(['Departement','SiteTheorique','eNodeBName','Secteur'], axis = 1)

print("Clean NULL data")
alldata_kpis = CleanNullCells(alldata_kpis)

Format data
Force data type
Remove unusefull structure
Clean NULL data


In [6]:
def get_config(local_fic_conf, local_type):
    
    # print(local_type)
    liste_retour = []
    fic_config = open(local_fic_conf, "r")
    fin = 0 
    
    for line in fic_config:
        # print(line)
        line = line.replace("\n", "")        
        if line.find(local_type)>=0 and fin == 0 :
            # print("trouve")
            for line in fic_config :
                if line != "\n" and fin==0:
                    line = line.replace("\n", "")
                    liste_retour.append(line)
                else:
                    # print("fin trouve")
                    fin = 1
    fic_config.close()
    return(liste_retour)

def extract_list_from_file(local_file):
    loc_file = open(local_file, "r")
    data_read=[]
    for line in loc_file:
        line = line.replace("\n", "")
        data_read.append(line)
    loc_file.close()
    
    return (data_read)



###################
# global variables
###################
local_path = 'C:/Users/euripab/Desktop/ML Extraction/'
config_file = "config_python_max_jour.txt"

config_group = get_config(local_path + "/" + config_file, "config_group")

list_group=[]
inventaire_groupe=[]
inventaire_selected_struct=[]
if len(config_group)==1 :
    # extract group configuration
    list_group = extract_list_from_file(local_path + "/" + config_group[0])
    groupe_definition = {}
    for groupe_def in list_group :
        groupe_def2 = groupe_def.split(";")
        inventaire_selected_struct.append(groupe_def2[1])
        if not groupe_def2[0] in inventaire_groupe :
            inventaire_groupe.append(groupe_def2[0])
        groupe_definition[groupe_def2[1]] = groupe_def2[0]
        
# pour extraire les donnees de chaque groupe
# passer de liste a dataframe ()
array_list_group=[]
# pb : on garde les entetes dans les donnees -> a corriger
# les colonnes du dataframe ne sontpas nommees
for item in list_group:
    split_item = item.split(";")
    array_list_group.append(split_item)

df_list_group = pd.DataFrame(array_list_group)

# extraction des donnees du groupe residentiel
# df_list_group.loc[df_list_group[0]=='residentiel'][1]
# ResCellsKpi = alldata_kpis.loc[alldata_kpis['CellName'].isin(df_list_group.loc[df_list_group[0]=='residentiel'][1])]

# enlever 'TDC4GData', absent des donnees sources dans certains fichiers extraits de Ipod LTE
# kpiList = ['CQImoyen','TDC4GData','TrafficDataDL','DemandesVoix4G',
#            'VoIPqualitymeasinsuffstat','NombredeConnexionsPS','nbUEVoIPqualityOK',
#            'Nbrtentatives_QCi5__TEAVoix','TraficDataUL']
    
kpiList = ['CQImoyen','TraficDataDL','TraficDataUL',
           'VoIPqualitymeasinsuffstat','CROZON_NombredeConnexionsPS','nbUEVoIPqualityOK',
           'Nbrtentatives_QCi5__TEAVoix']
        

In [7]:
#computing_type = "single" 
computing_type = "parallel"
list_type_cell=['L', 'T', 'D', 'P']
fic_data = local_path + "/data/" + 'all' + ".xlsx"
writer_data = pd.ExcelWriter(fic_data)
alldata_kpis.to_excel(writer_data, sheet_name="data")
for current_groupe in inventaire_groupe:
    # pour extraire les cellules du groupe residentiel
    print("extract data,group : {}".format(current_groupe))
    df_list_group.loc[df_list_group[0]==current_groupe][1]
    
    # extraction des donnees du groupe en cours d'analyse
    ZoneCellsKpi = alldata_kpis.loc[alldata_kpis['CellName'].isin(df_list_group.loc[df_list_group[0]==current_groupe][1])]
    fic_data = local_path + "/data/" + current_groupe + ".xlsx"
    writer_data = pd.ExcelWriter(fic_data)
    ZoneCellsKpi.to_excel(writer_data, sheet_name="data")
    writer_data.save()
    writer_data.close()
    
    for type_cell in list_type_cell:
        ZoneCellTypeKpi = ZoneCellsKpi.loc[ZoneCellsKpi['CellName'].str.startswith(type_cell)]
        fic_data = local_path + "/data/" + current_groupe + "_" + type_cell + ".xlsx"
        writer_data = pd.ExcelWriter(fic_data)        
        ZoneCellTypeKpi.to_excel(writer_data, sheet_name="data")
        writer_data.save()
        writer_data.close()
        


extract data,group : residentiel
extract data,group : Gare_St_lazare
extract data,group : Chatelet_les_halles
extract data,group : rive_droite_seine
extract data,group : concorde


In [9]:
computing_type = "single"
liste_data = os.listdir(local_path + "/data/")
time_struct={}
last_group = ''
for current_data in liste_data:
    current_group = current_data.replace("_D.xlsx", "")
    current_group = current_group.replace("_L.xlsx", "")
    current_group = current_group.replace("_T.xlsx", "")
    current_group = current_group.replace("_P.xlsx", "")
    current_group = current_group.replace(".xlsx", "")
    if current_group != last_group:
        time_struct={}
    last_group = current_group    
    ZoneCellsKpi=pd.read_excel(local_path + "/data/" + current_data)
    fic_result= local_path + "/KSTest/results1_Pvalues/" + "KSTest_" + current_data 
    fic_time= local_path + "/KSTest/compute_time/" + "time_KS_" + current_group + ".xlsx"
    writer_results = pd.ExcelWriter(fic_result)
    writer_time = pd.ExcelWriter(fic_time)
    if computing_type == 'single':
        time_struct[current_data]={}
        print("Compute KS_Single fichier : {}, groupe : {})".format(current_data, current_group))
        for kpi in kpiList:
            time_struct[current_data][kpi]={}
            time_struct[current_data][kpi]["delta_t"]=[]
            start_time = time.time()
            KpiPvalueMatrix = KSTestAllCellsPerDayAverage(ZoneCellsKpi,kpi)
            end_time = time.time()
            time_struct[current_data][kpi]["delta_t"].append(end_time-start_time)                
            KpiPvalueMatrix.to_excel(writer_results,sheet_name=kpi)
            df_time_struct = pd.DataFrame(time_struct)
            df_time_struct.to_excel(writer_time,sheet_name=current_group)
        writer_time.save()
        writer_results.save()

    elif computing_type == 'parallel':
        time_struct[current_data]={}
        print("Compute KS_parallel fichier : {}".format(current_data))
        for kpi in kpiList:
            time_struct[current_data][kpi]={}
            time_struct[current_data][kpi]["delta_t"]=[]
            start_time = time.time()
            results = KSTestAllCellsPerDayAverageParallel(ZoneCellsKpi,kpi)
            end_time = time.time()  
            KpiPvalueMatrix = buildKSTESTMatrixFromMPTEST(results)
            del results
            time_struct[current_data][kpi]["delta_t"].append(end_time-start_time)                 
            KpiPvalueMatrix.to_excel(writer_results,sheet_name=kpi)
            df_time_struct = pd.DataFrame(time_struct)
            df_time_struct.to_excel(writer_time,sheet_name=current_group)
        writer_time.save()
        writer_results.save()

Compute KS_Single fichier : Chatelet_les_halles.xlsx, groupe : Chatelet_les_halles)


KeyboardInterrupt: 

In [8]:
liste_data = os.listdir(local_path + "/KSTest/results3_PvaluesDays/")
time_struct={}
last_group = ''
for current_data in liste_data:
    current_group = current_data.replace("_D.xlsx", "")
    current_group = current_group.replace("_L.xlsx", "")
    current_group = current_group.replace("_T.xlsx", "")
    current_group = current_group.replace("_P.xlsx", "")
    current_group = current_group.replace(".xlsx", "")
    if current_group != last_group:
        time_struct={}
    last_group = current_group    

    fic_result= local_path + "/KSTest/results4_ClustersDays/" + "clustering_" + current_data
    fic_time= local_path + "/KSTest/compute_time/" + "time_clustering_" + current_group + ".xlsx"
    writer_results = pd.ExcelWriter(fic_result)
    writer_time = pd.ExcelWriter(fic_time)
    print("Compute Clustering fichier : {}, groupe : {})".format(current_data, current_group[:30]))
    path = local_path + "/KSTest/results3_PvaluesDays/" +current_data
    output = generateKpiLabeledDataFromExcelKSTEST(path, kpiList)
    time_struct[current_data]=output[1]
    output[0].to_excel(writer_results,sheet_name=current_group[:30])
    df_time_struct = pd.DataFrame(time_struct)
    df_time_struct.to_excel(writer_time,sheet_name=current_group[:30])
    writer_time.save()
    writer_results.save()


Compute Clustering fichier : KSTest_2018-08-07_Chatelet_les_halles.xlsx, groupe : KSTest_2018-08-07_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-07_Chatelet_les_halles_D.xlsx, groupe : KSTest_2018-08-07_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-07_Chatelet_les_halles_L.xlsx, groupe : KSTest_2018-08-07_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-07_Chatelet_les_halles_P.xlsx, groupe : KSTest_2018-08-07_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-07_Chatelet_les_halles_T.xlsx, groupe : KSTest_2018-08-07_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-07_concorde.xlsx, groupe : KSTest_2018-08-07_concorde)
Compute Clustering fichier : KSTest_2018-08-07_concorde_D.xlsx, groupe : KSTest_2018-08-07_concorde)
Compute Clustering fichier : KSTest_2018-08-07_concorde_L.xlsx, groupe : KSTest_2018-08-07_concorde)
Compute Clustering fichier : KSTest_2018-08-07_concorde_P.xlsx, groupe : KSTest_2018-08-07_concorde)
Compute Clustering f

Compute Clustering fichier : KSTest_2018-08-10_Chatelet_les_halles.xlsx, groupe : KSTest_2018-08-10_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-10_Chatelet_les_halles_D.xlsx, groupe : KSTest_2018-08-10_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-10_Chatelet_les_halles_L.xlsx, groupe : KSTest_2018-08-10_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-10_Chatelet_les_halles_P.xlsx, groupe : KSTest_2018-08-10_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-10_Chatelet_les_halles_T.xlsx, groupe : KSTest_2018-08-10_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-10_concorde.xlsx, groupe : KSTest_2018-08-10_concorde)
Compute Clustering fichier : KSTest_2018-08-10_concorde_D.xlsx, groupe : KSTest_2018-08-10_concorde)
Compute Clustering fichier : KSTest_2018-08-10_concorde_L.xlsx, groupe : KSTest_2018-08-10_concorde)
Compute Clustering fichier : KSTest_2018-08-10_concorde_P.xlsx, groupe : KSTest_2018-08-10_concorde)
Compute Clustering f

Compute Clustering fichier : KSTest_2018-08-13_Chatelet_les_halles.xlsx, groupe : KSTest_2018-08-13_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-13_Chatelet_les_halles_D.xlsx, groupe : KSTest_2018-08-13_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-13_Chatelet_les_halles_L.xlsx, groupe : KSTest_2018-08-13_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-13_Chatelet_les_halles_P.xlsx, groupe : KSTest_2018-08-13_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-13_Chatelet_les_halles_T.xlsx, groupe : KSTest_2018-08-13_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-13_concorde.xlsx, groupe : KSTest_2018-08-13_concorde)
Compute Clustering fichier : KSTest_2018-08-13_concorde_D.xlsx, groupe : KSTest_2018-08-13_concorde)
Compute Clustering fichier : KSTest_2018-08-13_concorde_L.xlsx, groupe : KSTest_2018-08-13_concorde)
Compute Clustering fichier : KSTest_2018-08-13_concorde_P.xlsx, groupe : KSTest_2018-08-13_concorde)
Compute Clustering f

Compute Clustering fichier : KSTest_2018-08-16_Chatelet_les_halles.xlsx, groupe : KSTest_2018-08-16_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-16_Chatelet_les_halles_D.xlsx, groupe : KSTest_2018-08-16_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-16_Chatelet_les_halles_L.xlsx, groupe : KSTest_2018-08-16_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-16_Chatelet_les_halles_P.xlsx, groupe : KSTest_2018-08-16_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-16_Chatelet_les_halles_T.xlsx, groupe : KSTest_2018-08-16_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-16_concorde.xlsx, groupe : KSTest_2018-08-16_concorde)
Compute Clustering fichier : KSTest_2018-08-16_concorde_D.xlsx, groupe : KSTest_2018-08-16_concorde)
Compute Clustering fichier : KSTest_2018-08-16_concorde_L.xlsx, groupe : KSTest_2018-08-16_concorde)
Compute Clustering fichier : KSTest_2018-08-16_concorde_P.xlsx, groupe : KSTest_2018-08-16_concorde)
Compute Clustering f

Compute Clustering fichier : KSTest_2018-08-19_Chatelet_les_halles.xlsx, groupe : KSTest_2018-08-19_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-19_Chatelet_les_halles_D.xlsx, groupe : KSTest_2018-08-19_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-19_Chatelet_les_halles_L.xlsx, groupe : KSTest_2018-08-19_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-19_Chatelet_les_halles_P.xlsx, groupe : KSTest_2018-08-19_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-19_Chatelet_les_halles_T.xlsx, groupe : KSTest_2018-08-19_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-19_concorde.xlsx, groupe : KSTest_2018-08-19_concorde)
Compute Clustering fichier : KSTest_2018-08-19_concorde_D.xlsx, groupe : KSTest_2018-08-19_concorde)
Compute Clustering fichier : KSTest_2018-08-19_concorde_L.xlsx, groupe : KSTest_2018-08-19_concorde)
Compute Clustering fichier : KSTest_2018-08-19_concorde_P.xlsx, groupe : KSTest_2018-08-19_concorde)
Compute Clustering f

...

Compute Clustering fichier : KSTest_2018-08-25_Chatelet_les_halles.xlsx, groupe : KSTest_2018-08-25_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-25_Chatelet_les_halles_D.xlsx, groupe : KSTest_2018-08-25_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-25_Chatelet_les_halles_L.xlsx, groupe : KSTest_2018-08-25_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-25_Chatelet_les_halles_P.xlsx, groupe : KSTest_2018-08-25_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-25_Chatelet_les_halles_T.xlsx, groupe : KSTest_2018-08-25_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-25_concorde.xlsx, groupe : KSTest_2018-08-25_concorde)
Compute Clustering fichier : KSTest_2018-08-25_concorde_D.xlsx, groupe : KSTest_2018-08-25_concorde)
Compute Clustering fichier : KSTest_2018-08-25_concorde_L.xlsx, groupe : KSTest_2018-08-25_concorde)
Compute Clustering fichier : KSTest_2018-08-25_concorde_P.xlsx, groupe : KSTest_2018-08-25_concorde)
Compute Clustering f

Compute Clustering fichier : KSTest_2018-08-28_Chatelet_les_halles.xlsx, groupe : KSTest_2018-08-28_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-28_Chatelet_les_halles_D.xlsx, groupe : KSTest_2018-08-28_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-28_Chatelet_les_halles_L.xlsx, groupe : KSTest_2018-08-28_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-28_Chatelet_les_halles_P.xlsx, groupe : KSTest_2018-08-28_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-28_Chatelet_les_halles_T.xlsx, groupe : KSTest_2018-08-28_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-28_concorde.xlsx, groupe : KSTest_2018-08-28_concorde)
Compute Clustering fichier : KSTest_2018-08-28_concorde_D.xlsx, groupe : KSTest_2018-08-28_concorde)
Compute Clustering fichier : KSTest_2018-08-28_concorde_L.xlsx, groupe : KSTest_2018-08-28_concorde)
Compute Clustering fichier : KSTest_2018-08-28_concorde_P.xlsx, groupe : KSTest_2018-08-28_concorde)
Compute Clustering f

Compute Clustering fichier : KSTest_2018-08-31_Chatelet_les_halles.xlsx, groupe : KSTest_2018-08-31_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-31_Chatelet_les_halles_D.xlsx, groupe : KSTest_2018-08-31_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-31_Chatelet_les_halles_L.xlsx, groupe : KSTest_2018-08-31_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-31_Chatelet_les_halles_P.xlsx, groupe : KSTest_2018-08-31_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-31_Chatelet_les_halles_T.xlsx, groupe : KSTest_2018-08-31_Chatelet_les)
Compute Clustering fichier : KSTest_2018-08-31_concorde.xlsx, groupe : KSTest_2018-08-31_concorde)
Compute Clustering fichier : KSTest_2018-08-31_concorde_D.xlsx, groupe : KSTest_2018-08-31_concorde)
Compute Clustering fichier : KSTest_2018-08-31_concorde_L.xlsx, groupe : KSTest_2018-08-31_concorde)
Compute Clustering fichier : KSTest_2018-08-31_concorde_P.xlsx, groupe : KSTest_2018-08-31_concorde)
Compute Clustering f

Compute Clustering fichier : KSTest_2018-09-03_Chatelet_les_halles.xlsx, groupe : KSTest_2018-09-03_Chatelet_les)
Compute Clustering fichier : KSTest_2018-09-03_Chatelet_les_halles_D.xlsx, groupe : KSTest_2018-09-03_Chatelet_les)
Compute Clustering fichier : KSTest_2018-09-03_Chatelet_les_halles_L.xlsx, groupe : KSTest_2018-09-03_Chatelet_les)
Compute Clustering fichier : KSTest_2018-09-03_Chatelet_les_halles_P.xlsx, groupe : KSTest_2018-09-03_Chatelet_les)
Compute Clustering fichier : KSTest_2018-09-03_Chatelet_les_halles_T.xlsx, groupe : KSTest_2018-09-03_Chatelet_les)
Compute Clustering fichier : KSTest_2018-09-03_concorde.xlsx, groupe : KSTest_2018-09-03_concorde)
Compute Clustering fichier : KSTest_2018-09-03_concorde_D.xlsx, groupe : KSTest_2018-09-03_concorde)
Compute Clustering fichier : KSTest_2018-09-03_concorde_L.xlsx, groupe : KSTest_2018-09-03_concorde)
Compute Clustering fichier : KSTest_2018-09-03_concorde_P.xlsx, groupe : KSTest_2018-09-03_concorde)
Compute Clustering f

Compute Clustering fichier : KSTest_2018-09-06_Chatelet_les_halles.xlsx, groupe : KSTest_2018-09-06_Chatelet_les)
Compute Clustering fichier : KSTest_2018-09-06_Chatelet_les_halles_D.xlsx, groupe : KSTest_2018-09-06_Chatelet_les)
Compute Clustering fichier : KSTest_2018-09-06_Chatelet_les_halles_L.xlsx, groupe : KSTest_2018-09-06_Chatelet_les)
Compute Clustering fichier : KSTest_2018-09-06_Chatelet_les_halles_P.xlsx, groupe : KSTest_2018-09-06_Chatelet_les)
Compute Clustering fichier : KSTest_2018-09-06_Chatelet_les_halles_T.xlsx, groupe : KSTest_2018-09-06_Chatelet_les)
Compute Clustering fichier : KSTest_2018-09-06_concorde.xlsx, groupe : KSTest_2018-09-06_concorde)
Compute Clustering fichier : KSTest_2018-09-06_concorde_D.xlsx, groupe : KSTest_2018-09-06_concorde)
Compute Clustering fichier : KSTest_2018-09-06_concorde_L.xlsx, groupe : KSTest_2018-09-06_concorde)
Compute Clustering fichier : KSTest_2018-09-06_concorde_P.xlsx, groupe : KSTest_2018-09-06_concorde)
Compute Clustering f

In [10]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(name+'(t-%d)' % (i)) for name in data.columns]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [name+'(t)' for name in data.columns]
		else:
			names += [(name+'(t+%d)' % (i)) for name in data.columns]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg


In [16]:
groupe= 'Gare_St_lazare'
#Creating Clustering Supervised Data
computing_type = "single"
liste_data = os.listdir(local_path + "/KSTest/results5_ClusteringTimeSeries/")
allCellSupervisedData = pd.DataFrame()
predictive_time_units = 1
to_predict_time_units = 1
for current_data in liste_data:
    if groupe in current_data:
        timeSeries = pd.read_excel(local_path + "/KSTest/results5_ClusteringTimeSeries/" + current_data)
        cellSupervisedData = series_to_supervised(timeSeries, n_in=predictive_time_units, n_out=to_predict_time_units, dropnan=True)
        allCellSupervisedData = allCellSupervisedData.append(cellSupervisedData, ignore_index=True)
    
fic_result= local_path + "/KSTest/results6_ClusteringSupervisedData/" +'supervised_' +groupe+str(predictive_time_units)+'_'+str(to_predict_time_units)+'.xlsx'
writer_results = pd.ExcelWriter(fic_result)
allCellSupervisedData.to_excel(writer_results)
writer_results.save()

In [15]:
kpiList = ['CQImoyen','TraficDataDL','TraficDataUL',
           'VoIPqualitymeasinsuffstat','CROZON_NombredeConnexionsPS','nbUEVoIPqualityOK',
           'Nbrtentatives_QCi5__TEAVoix']
#Creating Clustering Time Series Data
computing_type = "single"
liste_data = os.listdir(local_path + "/KSTest/results4_ClustersDays/")
time_struct={}
last_group = ''
auxList = []
num = 0
groupe = 'Gare_St_lazare.xlsx'
for current_data in liste_data:
    if current_data.endswith(groupe): 
        ZoneCellsKpi = pd.read_excel(local_path + "/KSTest/results4_ClustersDays/" + current_data)
        for kpi in ZoneCellsKpi.columns:
            #ZoneCellsKpi[kpi] =len(ZoneCellsKpi)/ZoneCellsKpi.groupby(kpi)[kpi].transform('count')
            ZoneCellsKpi[kpi] = ZoneCellsKpi.groupby(kpi)[kpi].transform('count')/len(ZoneCellsKpi)
            ZoneCellsKpi[kpi] = ZoneCellsKpi[kpi]*ZoneCellsKpi[kpi].apply(lambda x: linearMap(x,0.8,ZoneCellsKpi[kpi].max(),ZoneCellsKpi[kpi].min()))
        ZoneCellsKpi.index.name = 'Cell'
        ZoneCellsKpi['Tec'] = [cell[0] for cell in ZoneCellsKpi.index]
        ZoneCellsKpi.columns = [str(col)+' '+str(num) for col in ZoneCellsKpi.columns]
        num = num+1
        auxList.append(ZoneCellsKpi)
alldaysClusters = functools.reduce(lambda left,right: pd.merge(left,right,on='Cell'), auxList)
kpiList.append('Tec')
for cell in alldaysClusters.index:
    cellAllKpi = pd.DataFrame()
    for kpi in kpiList:
        cellkpiSeries = alldaysClusters.loc[cell][alldaysClusters.columns[alldaysClusters.columns.str.startswith(kpi)]]
        cellAllKpi[kpi] = np.array(cellkpiSeries)
    fic_result= local_path + "/KSTest/results5_ClusteringTimeSeries/" +'clustering_ts_'+groupe.replace(".xlsx", "")+'_' +str(cell)+'.xlsx'
    writer_results = pd.ExcelWriter(fic_result)
    cellAllKpi.to_excel(writer_results)
    writer_results.save()


In [14]:
def linearMap(x,r,MAX,MIN):
    return((1-r)*(x-MAX)/(MAX-MIN)+1)
    